In [1]:
import numpy as np

### Variableクラスの実装

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data

In [3]:
data  =np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [4]:
x.data = 2.0
print(x.data)

2.0


In [5]:
x = np.array([[0,1,2],[1,2,3]])
x.ndim

2

### Functionクラスの実装

In [6]:
class Function:
    def __call__(self, input):
        x = input.data  # ここでデータを取り出す
        y = x**2
        output = Variable(y)  # Vriableをして返す
        return output
    
# 箱として受け取って箱として返す

In [7]:
x = Variable(np.array(10))  # Variableクラス
f = Function()  # Functionクラス
y = f(x)  # Variableクラスを入れてVariableクラスが帰ってくる
print(y.data)
print(type(y))

100
<class '__main__.Variable'>


In [8]:
!python --version

Python 3.7.4


### Functionクラスを基底クラスとして実装

In [9]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)  # 具体的な計算、各関数の計算はforwardで行う
        output = Variable(y)  # Variableクラスにして返す
        
        return output
    
    def forward(self, x):
        raise NotImplementedError()

### Functionクラスを継承して入力された値を二乗するクラスの実装

In [10]:
class Square(Function):
    def forward(self, x):  # forwardメソッドをオーバーライド
        return x**2
    

In [11]:
x = Variable(np.array(10))
f = Square()
y = f(x)

print(y.data)
print(type(y))

100
<class '__main__.Variable'>


### exp関数の実装

In [12]:
class Exp(Function):  # Function継承し忘れると、'Exp' object is not callable
    def forward(self, x):
        return np.exp(x)

### 関数の連結

In [13]:
# y = (exp(x**2))**2
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

print(y.data)

1.648721270700128


### 導関数の定義から数値微分の実装

In [14]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    
    return (y1.data - y0.data) / (2*eps)  # インスタンスから値を取り出して計算、返却

In [15]:
# y = x**2の　x=2 における数値微分を求める
f = Square()
x = Variable(np.array(2.0))

dy = numerical_diff(f, x)

print(type(dy))  # これはVariableインスタンスで返しているわけではない
print(dy)

<class 'numpy.float64'>
4.000000000004


### 合成関数の微分

In [16]:
# y = (exp(x**2))**2　の微分
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    # これらもオブジェクト
    
    return C(B(A(x)))

print(type(f))

<class 'function'>


In [17]:
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)  # Functionインスタンスと Variableインスタンスを入れている
print(dy)

3.2974426293330694


### Variableクラスの拡張

In [18]:

class Variable:
    def __init__(self, data):
        self.data = data   # 通常の値
        self.grad = None  # 対応する微分した値を持つように拡張

### Functionクラスの拡張
- 入力変数の保持
- backwardメソッドの定義

In [19]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input   # 入力された変数を覚える
        
        return output
    
    def forward(self, x):
        raise NotImplementalError()
        
    def backward(self, gy):
        raise NotImplementalError()
        

In [20]:
class Square(Function):
    def forward(self, x):  # forwardメソッドをオーバーライド
        return x**2
    
    def backward(self, gy):
        x = self.input.data   # ここで使うために Functionの__call__メソッドで関数が実行されたときにinputを保持している
        gx = 2*x*gy
        return gx

class Exp(Function):
    def forward(self, x):   # forwardは__call__の中で呼び出されるときにすでにinputから取り出したxを引数に与えられてる
        y = np.exp(x)
        return y
    
    def backward(self,gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx
        

### Backpropagationの実装

In [21]:
# 順伝播
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 逆伝播
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

# numerical_diffで求めたgradとほぼ同じ→高い確率で正しく実装できていることが推測される
# 勾配確認OK

3.297442541400256


### Backpropagationの自動化

In [22]:
class Variable:
    def __init__(self, data):
        self.data = data   # 通常の値
        self.grad = None  # 対応する微分した値を持つように拡張
        self.creator = None  # 生みの親（の関数）を記録できるように拡張
        
    def set_creator(self, func):  # 生みの親を記録するためのメソッド　順伝播のときに一緒に実行させる
        self.creator = func
    

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # 出力変数に生みの親を覚えさせる　　繋がり
        self.input = input   # 入力された変数を覚える　　繋がり
        self.output = output  # 出力した変数も覚える　　繋がり
        
        return output
    
    def forward(self, x):
        raise NotImplementalError()
        
    def backward(self, gy):
        raise NotImplementalError()
        


In [23]:
# これをもう一回やっておかないと前のversionのFunctionが記録された場所を見に行っていることになる？
class Square(Function):
    def forward(self, x):  # forwardメソッドをオーバーライド
        return x**2
    
    def backward(self, gy):
        x = self.input.data   # ここで使うために Functionの__call__メソッドで関数が実行されたときにinputを保持している
        gx = 2*x*gy
        return gx

class Exp(Function):
    def forward(self, x):   # forwardは__call__の中で呼び出されるときにすでにinputから取り出したxを引数に与えられてる
        y = np.exp(x)
        return y
    
    def backward(self,gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [24]:
# 順伝播　Functioインスタンスの__call__が実行されている
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(1.0))
a = A(x)
b = B(a)
y = C(b)

# 中を見に行って問題にすぐに対処できた！！
#print(y.creator)
#print(a.creator)
#print(b.creator)
#print(C.output)

# 逆向きに計算グラフのノードをたどる
# assertは　以降がTrueでない場合は例外が発生
assert y.creator == C
assert y.creator.input == b
assert y.creator.output == y
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

# 例外なし

### 逆伝播を試す

In [25]:
y.grad = np.array(1.0)  # gyを取得

C = y.creator  # 関数を取得
b = C.input  # 関数の入力を取得

b.grad = C.backward(y.grad)  # 関数のbackwardメソッドを呼ぶ


In [26]:
B.backward(b.grad)

14.778112197861299

In [27]:
A.backward(B.backward(C.backward(y.grad)))

29.556224395722598

In [28]:
print(a.grad)

None


### Variableクラスにbackwardメソッドの追加(再帰ver)

In [29]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
    # 変数にbackwardメソッドをもたせることで自動微分ができる    
    def backward(self):   
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)  # 関数のbackwardメソッドを呼び出す
            x.backward()  # 自分より前の変数のbackwardメソッドを呼ぶ（再帰）
        
    

In [30]:
# forward propagation
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# back propagation
y.grad = np.array(1.0)
y.backward()  # ここではbackprapagation実行しただけ、各変数のgradは格納されている
print(x.grad)
print(a.grad)
print(b.grad)

3.297442541400256
3.297442541400256
2.568050833375483


### Variableクラスにbackwardメソッドの追加(ループver)

In [39]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
    # 変数にbackwardメソッドをもたせることで自動微分ができる    
    def backward(self):   
        funcs = [self.creator]
        
        # funcをbufferとしてloopを回す
        while funcs:
            f = funcs.pop()  # 関数を末尾から取得
            x, y = f.input, f.output  # 関数の入出力
            x.grad = f.backward(y.grad)  # backwardメソッドを呼ぶ
            # print('OK')
            if x.creator is not None:
                funcs.append(x.creator)  # add the creator of the previous function to  the list
                
# この構造にすることで、計算グラフが枝分かれするような場合でもfuncに複数の関数が保持されるようになるのでは？


In [34]:
# forward propagation
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# back propagation
y.grad = np.array(1.0)
y.backward()  
print(x.grad)
print(a.grad)
print(b.grad)

# 前の結果と一緒

OK
OK
OK
3.297442541400256
3.297442541400256
2.568050833375483


### 関数を使いやすくする

In [35]:
def square(x):
    f = Square()
    return f(x)
# return Square()(x)  でも可

def exp(x):
    f = Exp()
    return f(x)
# return Exp()(x)  でも可

In [36]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y =  square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

OK
OK
OK
3.297442541400256


In [38]:
# 関数を連続して使えるようになる
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.grad = np.array(1.0)
y.backward()
print(x.grad)

OK
OK
OK
3.297442541400256


### backwardメソッドの簡略化

In [44]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
    # 変数にbackwardメソッドをもたせることで自動微分ができる    
    def backward(self):
        # y.grad = mp.array(1.0)の省略、データ型をself.gradとself.dataであわせる
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        
        # funcをbufferとしてloopを回す
        while funcs:
            f = funcs.pop()  # 関数を末尾から取得
            x, y = f.input, f.output  # 関数の入出力
            x.grad = f.backward(y.grad)  # backwardメソッドを呼ぶ
            # print('OK')
            if x.creator is not None:
                funcs.append(x.creator)  # add the creator of the previous function to  the list
                
# この構造にすることで、計算グラフが枝分かれするような場合でもfuncに複数の関数が保持されるようになるのでは？


In [45]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

# y.grad = np.array(1.0)  消してもできる！
y.backward()
print(x.grad)

3.297442541400256


### ndarrayだけを使う
Variableにndarray以外のもの（None）を入れたらエラー

In [48]:
class Variable:
    def __init__(self, data):
        # 例外処理を入れる　ndarray以外のものが入ったらTypeError
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        
        while funcs:
            f = funcs.pop()  
            x, y = f.input, f.output  
            x.grad = f.backward(y.grad)  
            
            if x.creator is not None:
                funcs.append(x.creator)  # add the creator of the previous function to  the list


In [49]:
Variable(np.array(1.0))
Variable(None)

Variable(1.0)  # ここで TypeError: <class 'float'> is not supported

TypeError: <class 'float'> is not supported

In [55]:
# 注意点 Numpyでは次元数がnp.arrayの0だと、データ型が変わってしまう
x = np.array([1.0])
y = x**2
print(type(x), x.ndim)
print(str(type(y)) +'\n')

x = np.array(1.0)
y = x**2
print(type(x), x.ndim)
print(str(type(y)) +'\n')

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>



In [56]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [62]:
print(np.isscalar(np.float64(1.0)))
print(np.isscalar(2.0))
print(np.isscalar(as_array(2.0)))
print(np.isscalar(np.array([1,2,3])))

True
True
False
False


In [65]:
print(type(as_array(2.0)))
print((as_array(2.0)).ndim)  # 0次元配列になった

print(type(as_array(2.0)))


<class 'numpy.ndarray'>
0


In [66]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))  #　これで、xが0次元でyがndarrayにならなくてもas_arrayでndarrayに戻してくれる
        output.set_creator(self)  
        self.input = input   
        self.output = output
        
        return output
    
    def forward(self, x):
        raise NotImplementalError()
        
    def backward(self, gy):
        raise NotImplementalError()
        
    

### pythonでユニットテスト

In [67]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Vriable(np.array(2.0))
        y = square(x)
        
    expected = np.array(4.0)
    self.assertEqual(y.data, expected)

NameError: name 'self' is not defined